In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold
import numpy as np # 벡터, 행렬 등 수치 연산을 수행하는 선형대수 라이브러리
import pandas as pd

In [2]:
data = pd.read_csv("./final_model_test_pred.csv")

In [2]:
# path_train = "./data/train_data_1.csv"

path0 = "./data/val_data_0.csv"
path1 = "./data/val_data_1.csv"
path2 = "./data/val_data_2.csv"
test_path = "./test_data.csv"

data_val0 = pd.read_csv(path0)
data_val1 = pd.read_csv(path1)
data_val2 = pd.read_csv(path2)

# data_train = pd.read_csv(path_train)
data_test = pd.read_csv(test_path)

# data_val = pd.read_csv(path1)
data_train = pd.concat([data_val0, data_val1,data_val2], ignore_index=False)

# data_train = pd.read_csv(path_train)
# data_val = pd.read_csv(path_val)
# data_test = pd.read_csv(path_val)

print(data_train.shape)
# print(data_val.shape)
print(data_test.shape)
# data_test.head(10)


(1946894, 58)
(3255254, 56)


In [3]:
data_test =data_test.drop(["Unnamed: 0"], axis=1)
data_test

,application_id,bank_id,product_id,loan_limit,loan_rate,is_applied,user_id,birth_year,gender,credit_score,...,purpose_5,purpose_6,purpose_7,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3
0,4,7,191,26000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
1,4,7,220,10000000.0,11.4,NaN,209531,1978.0,0.0,720.0,...,0,1,0,1,0,0,0,0,1,0
2,8,30,85,13000000.0,15.9,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
3,8,55,159,5000000.0,19.9,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
4,8,35,29,5000000.0,14.5,NaN,675741,1983.0,1.0,590.0,...,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3255249,2167778,42,258,3000000.0,14.4,NaN,340207,1989.0,1.0,700.0,...,0,1,0,1,0,0,0,0,1,0
3255250,2167791,35,29,5000000.0,12.8,NaN,485732,1971.0,1.0,680.0,...,0,0,0,1,0,0,0,0,1,0
3255251,2167822,10,149,10000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0
3255252,2167822,10,65,11000000.0,15.6,NaN,105164,1996.0,1.0,570.0,...,0,1,0,1,0,0,0,0,1,0


In [4]:
X_train = data_train.drop(["is_applied"], axis=1)
y_train = data_train["is_applied"]

# X_val = data_val.drop(["is_applied"], axis=1)
# y_val = data_val["is_applied"]

X_test = data_test.drop(["is_applied"], axis=1)
y_test = data_test["is_applied"]

# X_val = X_val.drop(["loanapply_insert_time", "insert_time", "company_enter_month"])
X_train = X_train.drop(["loanapply_insert_time"], axis=1)
X_train = X_train.drop(["insert_time"], axis=1)
X_train = X_train.drop(["company_enter_month"], axis=1)

# X_test = X_test.drop(["loanapply_insert_time"], axis=1)
# X_test = X_test.drop(["insert_time"], axis=1)
# X_test = X_test.drop(["company_enter_month"], axis=1)

# y_val = y_val.drop(["loanapply_insert_time", "insert_time", "company_enter_month"])


X_test.head(5)

,application_id,bank_id,product_id,loan_limit,loan_rate,user_id,birth_year,gender,credit_score,yearly_income,...,purpose_5,purpose_6,purpose_7,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3
0,4,7,191,26000000.0,11.4,209531,1978.0,0.0,720.0,26000000.0,...,0,1,0,1,0,0,0,0,1,0
1,4,7,220,10000000.0,11.4,209531,1978.0,0.0,720.0,26000000.0,...,0,1,0,1,0,0,0,0,1,0
2,8,30,85,13000000.0,15.9,675741,1983.0,1.0,590.0,9000000.0,...,0,0,0,1,0,0,0,0,1,0
3,8,55,159,5000000.0,19.9,675741,1983.0,1.0,590.0,9000000.0,...,0,0,0,1,0,0,0,0,1,0
4,8,35,29,5000000.0,14.5,675741,1983.0,1.0,590.0,9000000.0,...,0,0,0,1,0,0,0,0,1,0


In [5]:
X_train.head(5)

,application_id,bank_id,product_id,loan_limit,loan_rate,user_id,birth_year,gender,credit_score,yearly_income,...,insert_time_minute,personal_rehabilitation_yn_0,personal_rehabilitation_yn_1,personal_rehabilitation_yn_2,personal_rehabilitation_complete_yn_0,personal_rehabilitation_complete_yn_1,personal_rehabilitation_complete_yn_2,personal_rehabilitation_complete_yn_3,company_enter_month_year,company_enter_month_month
0,1867362.0,4.0,268.0,20000000.0,10.9,650628.0,1968.0,1.0,660.0,36000000.0,...,9,1,0,0,0,0,1,0,1970.0,1.0
1,1888879.0,59.0,150.0,14000000.0,17.6,75483.0,1974.0,0.0,640.0,30000000.0,...,22,1,0,0,0,0,1,0,1970.0,1.0
2,1950464.0,50.0,142.0,14000000.0,15.9,649072.0,1984.0,1.0,610.0,36000000.0,...,9,0,0,1,0,0,0,1,1970.0,1.0
3,642480.0,35.0,98.0,9000000.0,15.9,499722.0,1972.0,0.0,837.0,28000000.0,...,57,0,0,1,0,0,0,1,1970.0,1.0
4,1444962.0,21.0,196.0,3000000.0,12.5,126504.0,1992.0,1.0,690.0,29000000.0,...,35,1,0,0,0,0,1,0,1970.0,1.0


In [6]:
import pickle

with open('./RF_train_0.pkl', 'rb') as f:
    model_0 = pickle.load(f)
    
with open('./RF_train_1.pkl', 'rb') as f:
    model_1 = pickle.load(f)
    
with open('./RF_train_2.pkl', 'rb') as f:
    model_2 = pickle.load(f)

In [7]:
m0_pred_train = model_0.predict(X_train)
m1_pred_train = model_1.predict(X_train)
m2_pred_train = model_2.predict(X_train)

new_data_train = np.array([m0_pred_train,m1_pred_train,m2_pred_train])

# new_data_train.shape
new_data_train = np.transpose(new_data_train)
new_data_train.shape

(1946894, 3)

In [9]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

final_model_train = RandomForestClassifier(random_state=0)
final_model_train.fit(new_data_train, y_train)
final_model_train_pred = final_model_train.predict(new_data_train)

print(f1_score(y_train,final_model_train_pred))

0.8102274633184312


In [12]:
m0_pred_test = model_0.predict(X_test)
m1_pred_test = model_1.predict(X_test)
m2_pred_test = model_2.predict(X_test)

new_data_test = np.array([m0_pred_test, m1_pred_test ,m2_pred_test])

# new_data.shape
new_data_test = np.transpose(new_data_test)
new_data_test.shape

/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)
/Users/gimtaeyong/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


(3255254, 3)

In [14]:
final_model_test_pred = final_model_train.predict(new_data_test)

In [17]:
pd.DataFrame(final_model_test_pred).to_csv("./final_model_test_pred_2.csv")

In [18]:
data1 = pd.read_csv("./final_model_test_pred.csv")
data2 = pd.read_csv("./final_model_test_pred_2.csv")

In [19]:
data1.drop(["Unnamed: 0"], axis=1)

,Unnamed: 0,0
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
3255249,3255249,0.0
3255250,3255250,0.0
3255251,3255251,1.0
3255252,3255252,1.0


In [21]:
data2.drop(["Unnamed: 0"], axis=1)

,0
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3255249,0.0
3255250,0.0
3255251,1.0
3255252,1.0


In [22]:
data1.equals(data2)

True

In [23]:
data2.to_csv("./final_test_data.csv")